In [2]:
import torch
import hiddenlayer as hl
from model.vgg import VGG
from model.paf_model import PAFModel
from model.CPR import CPRmodel
import torch.nn as nn
import torchvision

In [3]:
def create_model(typ="cpr", backend = 'vgg'):
    if backend == 'vgg':
        backend = VGG()
        backend_feats = 128
    else:
        raise ValueError('Model ' + backend + ' not available.')
    model = PAFModel(backend,backend_feats,n_joints=18,n_paf=32,n_stages=7) if typ == 'paf' else \
                CPRmodel(backend,backend_feats,n_joints=18,n_paf=32,n_stages=7, blocktype='hg')

    return model

In [5]:
transforms = [
        hl.transforms.Fold("Conv > BatchNorm > Relu", "ConvBnRelu"),
        hl.transforms.Fold("Conv > BatchNorm", "ConvBn"),
        hl.transforms.Fold("""
            (BatchNorm > Relu > ConvBnRelu > ConvBnRelu > ConvBn) > Add > Relu
            """, "BottleneckBlock", "Bottleneck Block"),
        hl.transforms.FoldDuplicates()
    ]


In [ ]:
hl.build_graph(create_model("cpr").cuda(), torch.zeros([1, 3, 224, 224]).cuda(), transforms=transforms)

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to C:\Users\akimo/.torch\models\vgg19-dcbb9e9d.pth
  0%|          | 1204224/574673361 [00:11<1:42:17, 93435.96it/s] 